In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import os

import numpy as np
import pandas as pd
import sklearn
from tqdm.auto import tqdm

import scipy.stats
import sklearn.neighbors

from sklearn.model_selection import KFold, GridSearchCV

In [19]:

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test_masked.csv")

BEST ONE SO FAR : 0.74168 SCORE

In [22]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, StackingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


feature_cols = ["lat", "lon", "n_asset" , "asset_index", "urban", 'cluster_id', 'adm1dhs']
label_cols = ['water_index']
train_X = train_df[feature_cols]
train_Y = train_df[label_cols]
test_X = test_df[feature_cols]

# change all entries in the urban column from R to 1 and U to 0
train_X['urban'] = train_X['urban'].map({'R': 1, 'U': 0})
test_X['urban'] = test_X['urban'].map({'R': 1, 'U': 0})


# Define the base models to use in the ensemble
base_models = [
    # LinearRegression(),
    # BaggingRegressor(n_estimators=50, random_state=42),
    RandomForestRegressor(n_estimators=100, random_state=42),
    # GradientBoostingRegressor(n_estimators=50, random_state=42),
    # AdaBoostRegressor(n_estimators=50, random_state=42),
    # ExtraTreesRegressor(n_estimators=50, random_state=42),
    # VotingRegressor(estimators=[('rf', RandomForestRegressor(n_estimators=50, random_state=42)), ('gb', GradientBoostingRegressor(n_estimators=50, random_state=42)), ('ab', AdaBoostRegressor(n_estimators=50, random_state=42)), ('et', ExtraTreesRegressor(n_estimators=50, random_state=42))])
]

# Train each base model on the training data and make predictions on the testing data
base_model_predictions = []
for model in base_models:
    model.fit(train_X, train_Y)
    base_model_predictions.append(model.predict(test_X))
    
    
# get the mean of all the predictions in one array 
ensemble_predictions = np.zeros(len(base_model_predictions[0]))
for i in range(len(base_model_predictions)):
    if i == 0:
        ensemble_predictions = base_model_predictions[i]
    else:
        ensemble_predictions += base_model_predictions[i]

ensemble_predictions = ensemble_predictions / len(base_model_predictions)


solution_df = test_df.copy()
solution_df["water_index"] = ensemble_predictions
filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]
filtered_solution_df.to_csv("solution_ensemble.csv", index=False)

C:\Users\barya\AppData\Local\Temp\ipykernel_21756\148869694.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X['urban'] = train_X['urban'].map({'R': 1, 'U': 0})
C:\Users\barya\AppData\Local\Temp\ipykernel_21756\148869694.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_X['urban'] = test_X['urban'].map({'R': 1, 'U': 0})
C:\Users\barya\AppData\Local\Temp\ipykernel_21756\148869694.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

DNN IMPLEMENTATION BELOW (I trained it only on 100 images, so it's not very good.. i have 3090 gpu which i ran on 10k images, and it scored -1.8 lol)

In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.models import Model
from keras.backend import set_floatx


train_dataset = pd.read_csv('train.csv')

train_subset = train_dataset.sample(n=100, random_state=42)

validation_dataset = pd.read_csv('test_masked.csv')


training_images = []
for filepath in train_subset['path']:
    try:
        with np.load(filepath) as data:
            for key in data.keys():
                image_data = data[key]
    except:
        print("Error")
        image_data = np.zeros((255, 255, 8))
    training_images.append(image_data)

training_scores = train_subset['water_index'].values



validation_images = []
for filepath in validation_dataset['path']:
    try:
        with np.load(filepath) as data:
            for key in data.keys():
                image_data = data[key]
    except:
        print("Error")
        image_data = np.zeros((255, 255, 8))
    validation_images.append(image_data)
    
set_floatx('float32')
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

# Define the input shape
input_shape = (8, 255, 255)

# Define the input layer
inputs = Input(shape=input_shape)

# Define the convolutional layers
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# Flatten the output of the convolutional layers
flatten = Flatten()(pool3)

# Define the fully connected layers
fc1 = Dense(512, activation='relu')(flatten)
dropout1 = Dropout(0.5)(fc1)
fc2 = Dense(256, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(fc2)

# Define the output layer
output = Dense(1, activation='linear')(dropout2)

# Define the model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# Train the model
with tf.device('/GPU:0'):
    model.fit(np.array(training_images), np.array(training_scores), epochs=20, batch_size=16, validation_split=0.2)

# Generate some dummy data for prediction
x_pred = np.array(validation_images)


# Use the model to make predictions
with tf.device('/GPU:0'):
    y_pred = model.predict(x_pred)

solution_df = validation_dataset.copy()

solution_df["water_index"] = y_pred

filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]
filtered_solution_df.to_csv("solution_dnn.csv", index=False)

Epoch 1/20
5/5 [==============================] - 1s 105ms/step - loss: 38483.3633 - mse: 38483.3633 - val_loss: 1286.4202 - val_mse: 1286.4202
Epoch 2/20
5/5 [==============================] - 0s 49ms/step - loss: 3362.5854 - mse: 3362.5854 - val_loss: 38.7236 - val_mse: 38.7236
Epoch 3/20
5/5 [==============================] - 0s 42ms/step - loss: 375.6102 - mse: 375.6102 - val_loss: 34.7393 - val_mse: 34.7393
Epoch 4/20
5/5 [==============================] - 0s 42ms/step - loss: 59.2288 - mse: 59.2288 - val_loss: 0.6323 - val_mse: 0.6323
Epoch 5/20
5/5 [==============================] - 0s 52ms/step - loss: 22.6402 - mse: 22.6402 - val_loss: 9.5581 - val_mse: 9.5581
Epoch 6/20
5/5 [==============================] - 0s 36ms/step - loss: 12.0681 - mse: 12.0681 - val_loss: 2.9904 - val_mse: 2.9904
Epoch 7/20
5/5 [==============================] - 0s 42ms/step - loss: 4.4532 - mse: 4.4532 - val_loss: 2.4670 - val_mse: 2.4670
Epoch 8/20
5/5 [==============================] - 0s 35ms/step

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.